# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
Arxiv has 73 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2406.12025


extracting tarball to tmp_2406.12025...

 done.
Retrieving document from  https://arxiv.org/e-print/2406.12819


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2097/1211882699.py:51: LatexWarning: 2406.12025 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


extracting tarball to tmp_2406.12819...

 done.
  0: tmp_2406.12819/main.tex, 618 lines
  1: tmp_2406.12819/aassymbols.tex, 579 lines


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2406.12819/main.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


M. Benisty  ->  M. Benisty  |  ['M. Benisty']


Found 56 bibliographic references in tmp_2406.12819/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.12819-b31b1b.svg)](https://arxiv.org/abs/2406.12819) | **A Dust-Trapping Ring in the Planet-Hosting Disk of Elias 2-24**  |
|| A. S. Carvalho, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2024-06-20*|
|*Comments*| *18 pages, 12 figures, accepted to ApJ*|
|**Abstract**|            Rings and gaps are among the most widely observed forms of substructure in protoplanetary disks. A gap-ring pair may be formed when a planet carves a gap in the disk, which produces a local pressure maximum following the gap that traps inwardly drifting dust grains and appears as a bright ring due to the enhanced dust density. A dust-trapping ring would provide a promising environment for solid growth and possibly planetesimal production via the streaming instability. We present evidence of dust trapping in the bright ring of the planet-hosting disk Elias 2-24, from the analysis of 1.3 mm and 3 mm ALMA observations at high spatial resolution (0.029 arcsec, 4.0 au). We leverage the high spatial resolution to demonstrate that larger grains are more efficiently trapped and place constraints on the local turbulence ($8 \times 10^{-4} < \alpha_\mathrm{turb} < 0.03$) and the gas-to-dust ratio ($\Sigma_g / \Sigma_d < 30$) in the ring. Using a scattering-included marginal probability analysis we measure a total dust disk mass of $M_\mathrm{dust} = 13.8^{+0.7}_{-0.5} \times 10^{-4} \ M_\odot$. We also show that at the orbital radius of the proposed perturber, the gap is cleared of material down to a flux contrast of 10$^{-3}$ of the peak flux in the disk.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.12025-b31b1b.svg)](https://arxiv.org/abs/2406.12025) | **A 260 pc resolution ALMA map of HCN(1-0) in the galaxy NGC 4321**  |
|| L. Neumann, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-06-20*|
|*Comments*| *18 pages, 9 figures, accepted for pub in A&A, Jun 13, 2024*|
|**Abstract**|            The star formation rate (SFR) is tightly connected to the amount of dense gas in molecular clouds. However, it is not fully understood how the relationship between dense molecular gas and star formation varies within galaxies and in different morphological environments. In this work, we study dense gas and star formation in the nearby spiral galaxy NGC 4321 to test how the amount of dense gas and its ability to form stars varies with environmental properties at 260 pc scales. We present new ALMA observations of HCN(1-0) line emission. Combined with existing CO(2-1) observations from ALMA, and H-alpha from MUSE, as well as F2100W from JWST to trace the SFR, we measure the HCN/CO line ratio, a proxy for the dense gas fraction and SFR/HCN, a proxy for the star formation efficiency of the dense gas. Towards the centre of the galaxy, HCN/CO systematically increases while SFR/HCN decreases, but these ratios stay roughly constant throughout the disc. Spiral arms, interarm regions, and bar ends show similar HCN/CO and SFR/HCN. On the bar, there is a significantly lower SFR/HCN at a similar HCN/CO. We conclude that the centres of galaxies show the strongest environmental influence on dense gas and star formation, suggesting either that clouds couple strongly to the surrounding pressure or that HCN is tracing more of the bulk molecular gas that is less efficiently converted into stars. On the contrary, across the disc of NGC 4321, where the ISM pressure is typically low, SFR/HCN does not show large variations (< 0.3 dex) in agreement with Galactic observations of molecular clouds. Despite the large variations across environments and physical conditions, HCN/CO is a good predictor of the mean molecular gas surface density at 260 pc scales.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2406.12819/./4-ProbToomStokes_ScatTrue.png', 'tmp_2406.12819/./MidplaneGasToDustDens_bothBands.png', 'tmp_2406.12819/./Elias24_bothProfs_noGaussian_Tb.png', 'tmp_2406.12819/./1-ProbProfiles_ScatTrue.png', 'tmp_2406.12819/./1-ProbProfiles_ScatTrue_q3p5.png']
copying  tmp_2406.12819/./4-ProbToomStokes_ScatTrue.png to _build/html/
copying  tmp_2406.12819/./MidplaneGasToDustDens_bothBands.png to _build/html/
copying  tmp_2406.12819/./Elias24_bothProfs_noGaussian_Tb.png to _build/html/
copying  tmp_2406.12819/./1-ProbProfiles_ScatTrue.png to _build/html/
copying  tmp_2406.12819/./1-ProbProfiles_ScatTrue_q3p5.png to _build/html/
exported in  _build/html/2406.12819.md
    + _build/html/tmp_2406.12819/./4-ProbToomStokes_ScatTrue.png
    + _build/html/tmp_2406.12819/./MidplaneGasToDustDens_bothBands.png
    + _build/html/tmp_2406.12819/./Elias24_bothProfs_noGaussian_Tb.png
    + _build/html/tmp_2406.12819/./1-ProbProfiles_ScatTrue.png
    + _build/html/tmp_2406.12819/./1-Pro

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# A Dust-Trapping Ring in the Planet-Hosting Disk of Elias 2-24

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.12819-b31b1b.svg)](https://arxiv.org/abs/2406.12819)<mark>Appeared on: 2024-06-20</mark> -  _18 pages, 12 figures, accepted to ApJ_

</div>
<div id="authors">

A. S. Carvalho, et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** Rings and gaps are among the most widely observed forms of substructure in protoplanetary disks. A gap-ring pair may be formed when a planet carves a gap in the disk, which produces a local pressure maximum following the gap that traps inwardly drifting dust grains and appears as a bright ring due to the enhanced dust density. A dust-trapping ring would provide a promising environment for solid growth and possibly planetesimal production via the streaming instability. We present evidence of dust trapping in the bright ring of the planet-hosting disk Elias 2-24, from the analysis of 1.3 mm and 3 mm ALMA observations at high spatial resolution (0.029 arcsec, 4.0 au). We leverage the high spatial resolution to demonstrate that larger grains are more efficiently trapped and place constraints on the local turbulence ( $8 \times 10^{-4} < \alpha_\mathrm{turb} < 0.03$ ) and the gas-to-dust ratio ( $\Sigma_g / \Sigma_d < 30$ ) in the ring. Using a scattering-included marginal probability analysis we measure a total dust disk mass of $M_\mathrm{dust} = 13.8^{+0.7}_{-0.5} \times 10^{-4}  M_\odot$ . We also show that at the orbital radius of the proposed perturber, the gap is cleared of material down to a flux contrast of 10 $^{-3}$ of the peak flux in the disk.

</div>

<div id="div_fig1">

<img src="tmp_2406.12819/./4-ProbToomStokes_ScatTrue.png" alt="Fig15.1" width="50%"/><img src="tmp_2406.12819/./MidplaneGasToDustDens_bothBands.png" alt="Fig15.2" width="50%"/>

**Figure 15. -** **Left:** The marginal probability distributions of the Toomre $Q$ profile (upper panel) and the Stokes number profile (lower panel) sampled from the distributions shown in Figure \ref{fig:DustProfiles}. The candidate companion orbit radius is shown with the green dashed line and the ring center is shown with the magenta dashed line. The solid blue line marks the values preferred by the overall best-fit model. The white dashed lines in the upper panel show the $Q = 1$ and $Q = 1.7$ lines. Assuming a gas-to-dust ratio of 100, there are many locations in the disk where $Q < 1.7$, implying the disk would be gravitationally unstable. **Right:** The range of midplane gas to dust ratios in B77 estimated in Section \ref{sec:GasToDust}. The blue and red shaded regions indicate the upper and lower limits from the maximum and minimum $\sqrt{\alpha_\mathrm{turb}/\mathrm{St}}$ values in Table \ref{tab:gaussFits} for the 1.3 mm (blue) and 3 mm (red) data. The lower limit on allowed Stokes number (vertical dark blue line) is obtained by requiring gravitational stability, since we do not see spirals in the disk. The lower limit on $\rho_g/\rho_d$(horizontal dark blue line) is obtained by requiring the $\Sigma_g > \Sigma_d$. The "allowed" values between these limits are marked by the green shaded region.      (*fig:Toomre*)

</div>
<div id="div_fig2">

<img src="tmp_2406.12819/./Elias24_bothProfs_noGaussian_Tb.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Top:** Deconvolved radial intensity profiles computed from the 1.3 mm and 3 mm visibilities, with 1-$\sigma$ uncertainties shown as the blue and red shaded regions respectively. The green dashed line shows the semi-major axis of the protoplanet candidate reported in [Jorquera, Pérez and Chauvin (2021)](), with the green shading showing the astrometric uncertainty in its position. The grey shading marks the gap location and the black dashed line marks the center of the ring, both as defined in [Huang, Andrews and Dullemond (2018)](). **Bottom:** The radial profile of the spectral index $\alpha$ computed from the profiles in the top panel. The black scalebar shows the resolution element of the spectral index profile, while the purple bar indicates the systematic uncertainty in the spectral index $\alpha$ due to ALMA's 10\% absolute flux-scale uncertainty. The vertical hashes mark the locations of the smaller gaps discussed in Section \ref{sec:substructure}. (*fig:frankProfiles*)

</div>
<div id="div_fig3">

<img src="tmp_2406.12819/./1-ProbProfiles_ScatTrue.png" alt="Fig14.1" width="50%"/><img src="tmp_2406.12819/./1-ProbProfiles_ScatTrue_q3p5.png" alt="Fig14.2" width="50%"/>

**Figure 14. -** Radial profiles of the dust properties in the disk around Elias 24. The left column shows our results for $p = 2.5$, while the right column shows $p = 3.5$. The orbital radius of the candidate companion from [Jorquera, Pérez and Chauvin (2021)]() is shown with a green dashed line and the ring location is shown with a magenta dashed line. **Top:** The dust surface density profile. Notice the increase in surface density at the ring location and the significant decrease in the gap. **Center:** The maximum grain size profile, which also increases at the ring location, indicating potential grain growth in the ring. **Bottom:** The temperature profile distributions. The blue dotted line shows the prior temperature profile given by Equation \ref{eq:tempProf}, with $1 \sigma$ Gaussian standard deviations shown as blue dashed lines. (*fig:DustProfiles*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.12819"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

98  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
